In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def imshow(title,img):
  plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
  plt.title(title)

In [3]:
face_dir = "/content/drive/MyDrive/0-myComputerVisionProjects/siamese_dataset"

subdirs = [subdir for subdir in os.listdir(face_dir) if os.path.isdir(os.path.join(face_dir, subdir))]

In [4]:
subdirs

['Depika',
 'messi',
 'Ranbir',
 'Nora',
 'Urvashi',
 'gigi_hadid',
 'bella_hadid',
 'lana_del_rey']

# Image pairs

In [5]:
size = 224

# Dictionary to store face images for each person
face_dict = {}

# Load face images for each person into the face_dict dictionary
for subdir in subdirs:
    face_dict[subdir] = []
    subdir_path = os.path.join(face_dir, subdir)
    for filename in os.listdir(subdir_path):
        image_path = os.path.join(subdir_path, filename)
        image = cv2.imread(image_path)
        image = image.astype('float32')
        image = cv2.resize(image,(size,size))
        image = image / 255.

        face_dict[subdir].append(image)

In [6]:
import random

def make_pairs():

    pairs = []
    labels = []

    # Same pairs
    same_pairs = []
    for person in face_dict.keys():
        for i in range(len(face_dict[person])):
            for j in range(i+1, len(face_dict[person])):
                same_pairs.append([face_dict[person][i], face_dict[person][j]])

    # Different pairs
    diff_pairs = []
    for i in range(len(subdirs)):
        for j in range(i+1, len(subdirs)):
            for k in range(len(face_dict[subdirs[i]])):
                for l in range(len(face_dict[subdirs[j]])):
                    diff_pairs.append([face_dict[subdirs[i]][k], face_dict[subdirs[j]][l]])

    # Sample the same and different pairs to have the same number of samples
    n_pairs = min(len(same_pairs), len(diff_pairs))
    same_pairs = random.sample(same_pairs, n_pairs)
    diff_pairs = random.sample(diff_pairs, n_pairs)

    # Combine same and different pairs and labels
    pairs = same_pairs + diff_pairs
    labels = [1]*n_pairs + [0]*n_pairs

    return (np.array(pairs), np.array(labels))


In [7]:
(pairs , labels) = make_pairs() 

In [8]:
print(pairs.shape)
print(labels.shape)

(870, 2, 224, 224, 3)
(870,)


In [9]:
count_1 = np.count_nonzero(labels == 1)
count_0 = np.count_nonzero(labels == 0)
print(count_1)
print(count_0)

435
435


In [ ]:
imshow('d',pairs[50,1])

In [ ]:
imshow('d',pairs[50,0])

In [12]:
labels[50]

1

In [13]:
from sklearn.model_selection import train_test_split

train_pairs , val_pairs , train_labels , val_labels = train_test_split(pairs ,labels , test_size = 0.3)

In [14]:
print(val_labels.shape)
print(val_pairs.shape)
print(train_labels.shape)
print(train_pairs.shape)

(261,)
(261, 2, 224, 224, 3)
(609,)
(609, 2, 224, 224, 3)


# Siamese

In [128]:
def contrastive_loss(y, preds, margin=1):
 # cast the true class label data type to the predicted class label data type 
 y = tf.cast(y, preds.dtype)
 # calculate the contrastive loss between the true labels and the predicted labels
 squaredPreds = K.square(preds)
 squaredMargin = K.square(K.maximum(margin - preds, 0))
 loss = 1-K.mean(y * squaredPreds + (1 - y) * squaredMargin) #1-
 return loss
 

In [129]:
from keras import backend as K

def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [130]:
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,ZeroPadding2D,GlobalAveragePooling2D
from keras.layers import Input, Lambda
import tensorflow as tf

In [131]:
IMG_SHAPE = (224,224,3)

# First network

Fine tuning VGG19 for sisters network

In [132]:
from tensorflow.keras.applications.vgg19 import preprocess_input

In [133]:
def tf_siamese_nn(shape, embedding=64, fineTune=False):
    inputs = tf.keras.layers.Input(shape)
    preprocess_fn = preprocess_input
    base_model = tf.keras.applications.vgg19.VGG19(input_shape=shape, include_top=False, weights='imagenet')
    
    if fineTune==False:
        base_model.trainable=False
    else:
        base_model.trainable = True
        # Fine-tune from this layer onwards
        fine_tune_at = len(base_model.layers)-int(len(base_model.layers)*.10)
# Freeze all the layers before the `fine_tune_at` layer
        for layer in base_model.layers[:fine_tune_at]:
          layer.trainable =  False
          
    x=base_model(inputs)
    x=tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs=tf.keras.layers.Dense(embedding)(x)
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [134]:
first_model=tf_siamese_nn(IMG_SHAPE,64 , True) 
first_model.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 global_average_pooling2d_6   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_8 (Dense)             (None, 64)                32832     
                                                                 
Total params: 20,057,216
Trainable params: 2,392,640
Non-trainable params: 17,664,576
_________________________________________________________________


In [135]:
img1 = tf.keras.layers.Input(shape=IMG_SHAPE)
img2 =  tf.keras.layers.Input( shape=IMG_SHAPE)
featureExtractor = tf_siamese_nn(IMG_SHAPE)
featsA = featureExtractor(img1)
featsB = featureExtractor(img2)

In [136]:
distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])

In [137]:
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(distance)
model = tf.keras.Model(inputs=[img1, img2], outputs=outputs)

In [ ]:
model.compile(loss=contrastive_loss, optimizer=Adam(lr = 0.001), metrics=["accuracy"])
model.summary()

In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping

# early_stop = EarlyStopping(monitor='val_accuracy', patience=2, verbose=1,restore_best_weights=True)

In [177]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(filepath='/content/drive/MyDrive/0-myComputerVisionProjects/my_model.h5', save_weights_only=True)


In [ ]:

history = model.fit(
    [train_pairs[:, 0], train_pairs[:, 1]], train_labels[:],
    validation_data=([val_pairs[:, 0], val_pairs[:, 1]], val_labels[:]),
    batch_size=8, #32
    epochs=100,
    shuffle = True,
    callbacks=[checkpoint_callback]
    )


In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and Validation Losses',size = 20)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training and Validation Accuracies',size = 20)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
# model.save('')

# Second network

In [123]:

def build_siamese_model(inputShape, embeddingDim=48): #embedding_dim = 48
	# specify the inputs for the feature extractor network
    inputs = Input(inputShape)
    # define the first set of CONV => RELU => POOL => DROPOUT layers
    x = Conv2D(128, (3, 3), padding="same", activation="relu")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)
    # second set of CONV => RELU => POOL => DROPOUT layers
    x = Conv2D(128, (3, 3), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    # third set of CONV => RELU => POOL => DROPOUT layers
    x = Conv2D(128, (3, 3), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    # prepare the final outputs
    pooledOutput = GlobalAveragePooling2D()(x)
    outputs = Dense(embeddingDim)(pooledOutput)
    # build the model

    model = Model(inputs, outputs)
    # return the model to the calling function
    return model

In [60]:
IMG_SHAPE = (size,size,3)

In [124]:
imgA = Input(shape=IMG_SHAPE)
imgB = Input(shape=IMG_SHAPE)
featureExtractor = build_siamese_model(IMG_SHAPE)
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)
# finally, construct the siamese network
distance = Lambda(euclidean_distance)([featsA, featsB])
siamese_model = Model(inputs=[imgA, imgB], outputs=distance)

In [ ]:
siamese_model.compile(loss=contrastive_loss, optimizer=Adam(lr=0.001), metrics=["accuracy"])
siamese_model.summary()

In [ ]:
history_2 = siamese_model.fit(
    [train_pairs[:, 0], train_pairs[:, 1]], train_labels,
    validation_data=([val_pairs[:, 0], val_pairs[:, 1]], val_labels),
    batch_size=8, 
    epochs=100
    )

In [ ]:
# siamese_model.save('/content/drive/MyDrive/0-myComputerVisionProjects/siamese.h5')

# Third model

In [19]:
from keras.models import load_model
my_model = load_model('/content/drive/MyDrive/0-myComputerVisionProjects/face.h5')

In [ ]:
my_model.summary()

In [20]:
def siamese(shape,embedding=128):
  # Get the first 3 layers from model1
  layers_to_copy = my_model.layers[:4]

  # Create a new model with the same input as model2
  input_layer = my_model.input
  output = input_layer

  # Add the first 4 layers from my_model to the new model
  for layer in layers_to_copy:
      output = layer(output)

  # Create a new model with the copied layers and the remaining layers of model2
  new_model = Model(inputs=input_layer, outputs=output)
  for layer in new_model.layers:
        layer.trainable = False
  inputs = tf.keras.layers.Input(shape)

  x = new_model(inputs)

  x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = tf.keras.layers.MaxPool2D(2)(x)
  x = tf.keras.layers.Dropout(0.3)(x)

  x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = tf.keras.layers.MaxPool2D(2)(x)
  x = tf.keras.layers.Dropout(0.3)(x)

  x = tf.keras.layers.GlobalAveragePooling2D()(x)

  outputs=tf.keras.layers.Dense(embedding)(x)
  model = tf.keras.Model(inputs, outputs)
  return model

In [ ]:
third_model = siamese(IMG_SHAPE,128) #64
third_model.summary()

In [22]:
img1 = tf.keras.layers.Input(shape=IMG_SHAPE)
img2 =  tf.keras.layers.Input( shape=IMG_SHAPE)
featureExtractor = siamese(IMG_SHAPE)
featsA = featureExtractor(img1)
featsB = featureExtractor(img2)

In [23]:
distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])

In [24]:
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(distance)
final_model = tf.keras.Model(inputs=[img1, img2], outputs=outputs)

In [ ]:
final_model.summary()

Fine tuning

In [ ]:
# def siamese_nn(shape, embedding=128): #fineTune =  False
#     inputs = tf.keras.layers.Input(shape)
#     base_model = my_model
#     # Freeze all layers in the pre-trained model
#     for layer in base_model.layers[:-1]:
#        layer.trainable = False
#     # Add AverageGlobalPooling layer
#     x=base_model(inputs)
#     # x = base_model.output
#     # x=tf.keras.layers.GlobalAveragePooling2D()(x)

#     outputs=tf.keras.layers.Dense(embedding)(x)
#     model = tf.keras.Model(inputs, outputs)
    
#     return model

In [26]:
final_model.compile(loss="binary_crossentropy",optimizer=Adam(0.001),metrics=['accuracy'])

In [ ]:
final_model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_accuracy', patience=2, verbose=1,restore_best_weights=True)

In [27]:
h = final_model.fit(
    [train_pairs[:, 0], train_pairs[:, 1]], train_labels,
    validation_data=([val_pairs[:, 0], val_pairs[:, 1]], val_labels),
    batch_size=8, 
    epochs=100,
    shuffle=True
)

Epoch 1/100
77/77 [==============================] - 65s 832ms/step - loss: 0.6908 - accuracy: 0.5041 - val_loss: 0.6895 - val_accuracy: 0.5019
Epoch 2/100
77/77 [==============================] - 64s 841ms/step - loss: 0.6861 - accuracy: 0.5057 - val_loss: 0.6841 - val_accuracy: 0.4943
Epoch 3/100
77/77 [==============================] - 61s 796ms/step - loss: 0.6485 - accuracy: 0.5090 - val_loss: 0.5911 - val_accuracy: 0.6169
Epoch 4/100
77/77 [==============================] - 62s 806ms/step - loss: 0.5843 - accuracy: 0.6355 - val_loss: 0.5743 - val_accuracy: 0.6897
Epoch 5/100
77/77 [==============================] - 63s 820ms/step - loss: 0.5706 - accuracy: 0.7028 - val_loss: 0.5601 - val_accuracy: 0.7778
Epoch 6/100
77/77 [==============================] - 63s 818ms/step - loss: 0.5560 - accuracy: 0.7471 - val_loss: 0.5528 - val_accuracy: 0.7280
Epoch 7/100
77/77 [==============================] - 65s 845ms/step - loss: 0.5408 - accuracy: 0.7685 - val_loss: 0.5354 - val_accuracy:

In [ ]:
tf.saved_model.save(final_model, '/content/drive/MyDrive/0-myComputerVisionProjects/my_saved_siamese')

In [194]:
# load the model for prediction
loaded_model = tf.saved_model.load('/content/drive/MyDrive/0-myComputerVisionProjects/my_saved_siamese')

In [ ]:
# loaded = load_model('/content/final_model_weights.h5')

In [211]:
height,width = size,size
image1 = cv2.imread('/content/bela1.png')
image2 = cv2.imread('/content/bela2.png')

image1 = cv2.resize(image1,(height,width))
image2 = cv2.resize(image2,(height,width))

image1 = image1.astype('float32') / 255.0
image2 = image2.astype('float32') / 255.0

image1 = np.expand_dims(image1, axis=0)
image2 = np.expand_dims(image2, axis=0)

In [212]:
y_pred = final_model.predict([image1,image2])[0][0]

1/1 [==============================] - 0s 109ms/step


In [213]:
y_pred

0.9527568